# Patronage Calculation Tests
Wed 28 Aug 2018 - Tue 4 Sep 2018

In [1]:
import numpy as np
import pandas as pd

## Experiment with Timestamps in pandas

In [28]:
pd.Timestamp('1/1/2018')

Timestamp('2018-01-01 00:00:00')

In [29]:
pd.Timestamp('4pm 1/1/2018')

Timestamp('2018-01-01 16:00:00')

In [2]:
pd.Timestamp('1/1/2018') - pd.Timestamp('12/1/2017')

Timedelta('31 days 00:00:00')

In [3]:
(pd.Timestamp('1/1/2018') - pd.Timestamp('12/1/2017')).days / (pd.Timestamp('1/1/2018') - pd.Timestamp('1/1/2017')).days

0.08493150684931507

In [4]:
31/365

0.08493150684931507

In [5]:
np.floor(5.6)

5.0

## Create DataFrames with fake data to test functions for computing patronage

In [6]:
pd.DataFrame([[1,'a',3],[2,'b',5]], columns=['member_id','name','amount'])

,member_id,name,amount
0,1,a,3
1,2,b,5


In [7]:
contributions_df = pd.DataFrame([
    [1, "1/3/2017", 125, 'membership'],
    [2, "1/31/2017", 400, 'equity'],
    [1, "2/20/2017", 5000, 'preferred'],
    [1, "4/13/2017", 500, 'equity'],
    [2, "4/13/2017", 7500, 'preferred'],
    [3, "6/24/2017", 125, 'mebership'],
    [4, "9/17/2017", 3000, "preferred"],
    [3, "11/11/2017", 20000, 'preferred']
], columns = ['member_id','date','amount','type'])
contributions_df

,member_id,date,amount,type
0,1,1/3/2017,125,membership
1,2,1/31/2017,400,equity
2,1,2/20/2017,5000,preferred
3,1,4/13/2017,500,equity
4,2,4/13/2017,7500,preferred
5,3,6/24/2017,125,mebership
6,4,9/17/2017,3000,preferred
7,3,11/11/2017,20000,preferred


In [8]:
prev_equity_df = pd.DataFrame([
    [1, 'Alice', 0, 0],
    [2, 'Bob', 125, 0],
    [3, 'Darwin', 0, 0],
    [4, 'Eve', 225, 1000],
], columns = ['member_id', 'name', 'equity', 'preferred'])
prev_equity_df

,member_id,name,equity,preferred
0,1,Alice,0,0
1,2,Bob,125,0
2,3,Darwin,0,0
3,4,Eve,225,1000


In [9]:
contributions_df['member_id'].unique()

array([1, 2, 3, 4])

## Write and test a function to compute the fraction of the year remaining from a given date

In [10]:
def fraction_year_remaining(date):
    """Computes the fraction of the year remaining from a given date.
    """
    date = pd.to_datetime(date)
    offset = pd.tseries.offsets.YearBegin()
    year = date - offset
    next_year = date + offset
    #print(offset, year, next_year)
    return (next_year - date).days / (next_year - year).days

In [11]:
fraction_year_remaining('5/23/2017')

0.6109589041095891

In [12]:
fraction_year_remaining(pd.Timestamp('2/23/2017'))

0.8547945205479452

In [13]:
fraction_year_remaining('7/31/2017')

0.42191780821917807

In [14]:
type(pd.Timestamp('2/23/2017').year)

int

In [15]:
contributions_df['date'].apply(fraction_year_remaining)

0    0.994521
1    0.917808
2    0.863014
3    0.720548
4    0.720548
5    0.523288
6    0.290411
7    0.139726
Name: date, dtype: float64

## Use the "year fraction" function to compute new patronage for the year

In [16]:
contributions_df['amount']*contributions_df['date'].apply(fraction_year_remaining)

0     124.315068
1     367.123288
2    4315.068493
3     360.273973
4    5404.109589
5      65.410959
6     871.232877
7    2794.520548
dtype: float64

In [17]:
new_patronage_amounts = contributions_df['amount']*contributions_df['date'].apply(fraction_year_remaining)
temp = contributions_df.copy()
temp['patronage'] = new_patronage_amounts
temp

,member_id,date,amount,type,patronage
0,1,1/3/2017,125,membership,124.315068
1,2,1/31/2017,400,equity,367.123288
2,1,2/20/2017,5000,preferred,4315.068493
3,1,4/13/2017,500,equity,360.273973
4,2,4/13/2017,7500,preferred,5404.109589
5,3,6/24/2017,125,mebership,65.410959
6,4,9/17/2017,3000,preferred,871.232877
7,3,11/11/2017,20000,preferred,2794.520548


## Write and test functions to compute new patronage from DataFrames

In [18]:
def compute_transaction_patronage(contributions_df):
    """Compute patronage for each transaction based on amount and fraction of year remaining.
    """
    contributions_df = contributions_df.copy()
    patronage = contributions_df['amount']*contributions_df['date'].apply(fraction_year_remaining)
    contributions_df['patronage'] = patronage
    return contributions_df

def compute_new_patronage(contributions_df):
    """Compute each member's patronage from new contributions for the current year.
    """
    contributions_df = compute_transaction_patronage(contributions_df)
    return contributions_df[['member_id','patronage']].groupby(by='member_id').sum()

In [19]:
compute_transaction_patronage(contributions_df)

,member_id,date,amount,type,patronage
0,1,1/3/2017,125,membership,124.315068
1,2,1/31/2017,400,equity,367.123288
2,1,2/20/2017,5000,preferred,4315.068493
3,1,4/13/2017,500,equity,360.273973
4,2,4/13/2017,7500,preferred,5404.109589
5,3,6/24/2017,125,mebership,65.410959
6,4,9/17/2017,3000,preferred,871.232877
7,3,11/11/2017,20000,preferred,2794.520548


In [20]:
compute_new_patronage(contributions_df)

,patronage
member_id,
1,4799.657534
2,5771.232877
3,2859.931507
4,871.232877


## Experiment with setting the DataFrame index and renaming columns in pandas

In [21]:
prev_equity_df.set_index('member_id')

,name,equity,preferred
member_id,,,
1,Alice,0,0
2,Bob,125,0
3,Darwin,0,0
4,Eve,225,1000


In [22]:
temp = compute_transaction_patronage(contributions_df)
temp.rename(columns={'patronage': 'new_patronage'}, inplace=True)
temp

,member_id,date,amount,type,new_patronage
0,1,1/3/2017,125,membership,124.315068
1,2,1/31/2017,400,equity,367.123288
2,1,2/20/2017,5000,preferred,4315.068493
3,1,4/13/2017,500,equity,360.273973
4,2,4/13/2017,7500,preferred,5404.109589
5,3,6/24/2017,125,mebership,65.410959
6,4,9/17/2017,3000,preferred,871.232877
7,3,11/11/2017,20000,preferred,2794.520548


In [23]:
temp = prev_equity_df.set_index('member_id')
temp['new_patronage'] = compute_new_patronage(contributions_df)['patronage']
temp

,name,equity,preferred,new_patronage
member_id,,,,
1,Alice,0,0,4799.657534
2,Bob,125,0,5771.232877
3,Darwin,0,0,2859.931507
4,Eve,225,1000,871.232877


In [24]:
temp.rename(columns={'equity': 'prev_equity', 'preferred': 'prev_preferred'}, inplace=True)
temp

,name,prev_equity,prev_preferred,new_patronage
member_id,,,,
1,Alice,0,0,4799.657534
2,Bob,125,0,5771.232877
3,Darwin,0,0,2859.931507
4,Eve,225,1000,871.232877


## Write and test function to compute total patronage from existing equity and new equity DataFrames

In [41]:
def compute_patronage(prev_equity_df, contributions_df):
    """Compute total patronage for each member from new contributions for the current year
        and existing equity from previous years.
    """
    patronage_df = prev_equity_df.set_index('member_id')
    
    patronage_df['old_patronage'] = patronage_df['equity'] + patronage_df['preferred']
    patronage_df['new_patronage'] = compute_new_patronage(contributions_df)['patronage']
    patronage_df['patronage'] = patronage_df['old_patronage'] + patronage_df['new_patronage']
#     patronage_df.rename(columns={'equity': 'prev_equity', 'preferred': 'prev_preferred'}, inplace=True)
#     patronage_df['patronage'] = (
#         patronage_df['prev_equity'] + patronage_df['prev_preferred'] + patronage_df['new_patronage'])
    patronage_df['proportionate_patronage'] = patronage_df['patronage'] / patronage_df['patronage'].sum()
    return patronage_df[['name', 'old_patronage', 'new_patronage', 'patronage', 'proportionate_patronage']]

In [42]:
temp = compute_patronage(prev_equity_df, contributions_df)
temp

,name,old_patronage,new_patronage,patronage,proportionate_patronage
member_id,,,,,
1,Alice,0,4799.657534,4799.657534,0.306647
2,Bob,125,5771.232877,5896.232877,0.376707
3,Darwin,0,2859.931507,2859.931507,0.182719
4,Eve,1225,871.232877,2096.232877,0.133927


In [43]:
#Total 2017 patronage, used to compute the proportionate patronage for each member.
#This number is the time-weighted average amount of capital for the year 2017.
temp['patronage'].sum()

15652.054794520547